In [7]:

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import Ridge, Lasso
import xgboost as xgb
import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler


# Charger les données
data_train = pd.read_csv('data/data_train.csv', delimiter='\t', quotechar='"')

# Préparer les données pour l'imputation
data_not_null = data_train.dropna(subset=['G1', 'G2'])

# Modèle pour imputer G1 en utilisant G2
model_G1 = LinearRegression()
model_G1.fit(data_not_null[['G2']], data_not_null['G1'])

# Modèle pour imputer G2 en utilisant G1
model_G2 = LinearRegression()
model_G2.fit(data_not_null[['G1']], data_not_null['G2'])

# Imputer G1 pour les valeurs manquantes en utilisant G2
missing_G1 = data_train['G1'].isnull() & data_train['G2'].notnull()
data_train.loc[missing_G1, 'G1'] = model_G1.predict(data_train.loc[missing_G1, ['G2']])

# Imputer G2 pour les valeurs manquantes en utilisant G1
missing_G2 = data_train['G2'].isnull() & data_train['G1'].notnull()
data_train.loc[missing_G2, 'G2'] = model_G2.predict(data_train.loc[missing_G2, ['G1']])

# Supprimer les lignes avec des valeurs manquantes restantes
data_train = data_train.dropna(subset=['G1', 'G2'])



# Imputer les valeurs manquantes restantes avec la moyenne
# data_train['G1'].fillna(data_train['G1'].mean(), inplace=True)
# data_train['G2'].fillna(data_train['G2'].mean(), inplace=True)
data_train_cleaned = data_train

# Convertir les variables catégoriques en variables numériques (encodage one-hot)
data_train_encoded = pd.get_dummies(data_train_cleaned, drop_first=True)

In [14]:
data_train_encoded["address_U"].value_counts()

1    279
0    120
Name: address_U, dtype: int64

In [15]:


display(data_train_encoded.columns)

# Créer une nouvelle colonne pour la consommation totale d'alcool
data_train_encoded['Total_Alcohol_Consumption'] = data_train_encoded['Walc'] + data_train_encoded['Dalc']
data_train_encoded['Combined_G1_G2'] = data_train_encoded['G1'] + data_train_encoded['G2']
data_train_encoded['Combined_Parent_Education'] = data_train_encoded['Medu'] * data_train_encoded['Fedu']
data_train_encoded['Travel_Effect'] = data_train_encoded['address_U'] * data_train_encoded['traveltime']
data_train_encoded['Family_Dynamics'] = data_train_encoded['famsize_LE3'] * data_train_encoded['famrel']
# pas de sens
data_train_encoded['Study_Failures_Interaction'] = data_train_encoded['studytime'] * data_train_encoded['failures']
data_train_encoded['Absences_Health_Interaction'] = data_train_encoded['absences'] * data_train_encoded['health']
data_train_encoded['Study_Activities_Interaction'] = data_train_encoded['activities_yes'] * data_train_encoded['studytime']
data_train_encoded['Health_Alcohol_Interaction'] = data_train_encoded['health'] * data_train_encoded['Dalc']
# déja présent
data_train_encoded['Parent_Education'] = data_train_encoded['Medu'] + data_train_encoded['Fedu']

# Sélectionner manuellement les features basées sur l'analyse de corrélation
# On suppose ici que les colonnes d'intérêt sont encore présentes après encodage
### pas bon ca .
selected_features = [col for col in data_train_encoded.columns if col in ['G1', 'G2', 'studytime', 'Medu', 'Fedu', 'Walc', 'Dalc', 'Absences_Health_Interaction', 'Family_Dynamics', 'Health_Alcohol_Interaction', 'Travel_Effect']]

# Ajouter les nouvelles colonnes encodées manuellement si besoin
selected_features.extend([col for col in data_train_encoded.columns if col.startswith('school_') or col.startswith('sex_')])
X_encoded = data_train_encoded[selected_features]
y_encoded = data_train_encoded['G3']
print(len(X_encoded))
print(len(y_encoded))


Index(['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel',
       'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2',
       'G3', 'school_MS', 'sex_M', 'address_U', 'famsize_LE3', 'Pstatus_T',
       'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher',
       'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher',
       'reason_home', 'reason_other', 'reason_reputation', 'guardian_mother',
       'guardian_other', 'schoolsup_yes', 'famsup_yes', 'paid_yes',
       'activities_yes', 'nursery_yes', 'higher_yes', 'internet_yes',
       'romantic_yes'],
      dtype='object')

399
399


In [7]:
# Les modèles à tester

#GridSearchCV ou RandomizedSearchCV. pour trouver les hyper paramétre
models = {
    'Ridge': Ridge(
        alpha=1.0,                 # Paramètre de régularisation
        solver='auto',             # Méthode de résolution
        max_iter=None,             # Nombre maximal d'itérations
        random_state=None,         # Graîne aléatoire
        tol=1e-3                   # Tolérance pour la convergence
    ),
    
    'Lasso': Lasso(
        alpha=1.0,                 # Paramètre de régularisation
        max_iter=1000,             # Nombre maximal d'itérations
        tol=1e-4,                  # Tolérance pour la convergence
        random_state=None          # Graîne aléatoire
    ),
    
    'XGBoost': xgb.XGBRegressor(
        booster='gbtree',          # Type de booster ('gbtree', 'gblinear', 'dart')
        learning_rate=0.1,         # Taux d'apprentissage
        n_estimators=100,          # Nombre d'arbres
        max_depth=6,               # Profondeur maximale des arbres
        subsample=1,               # Proportion des données utilisées pour chaque arbre
        colsample_bytree=1,        # Proportion de colonnes échantillonnées par arbre
        random_state=0             # Graîne aléatoire
    ),
    
    'LightGBM': lgb.LGBMRegressor(
        boosting_type='gbdt',      # Type de boosting ('gbdt', 'dart', 'goss')
        learning_rate=0.1,         # Taux d'apprentissage
        n_estimators=100,          # Nombre d'arbres
        max_depth=-1,              # Profondeur maximale des arbres (-1 signifie pas de limite)
        subsample=1.0,             # Proportion des données utilisées pour chaque arbre
        colsample_bytree=1.0,      # Proportion de colonnes échantillonnées par arbre
        random_state=None          # Graîne aléatoire
    ),
    
    'Random Forest': RandomForestRegressor(
        n_estimators=100,          # Nombre d'arbres dans la forêt
        max_depth=None,            # Profondeur maximale des arbres
        min_samples_split=2,       # Nombre minimal d'échantillons pour diviser un nœud
        min_samples_leaf=1,        # Nombre minimal d'échantillons dans une feuille
        random_state=None,         # Graîne aléatoire
        n_jobs=-1                  # Utilisation de tous les cœurs disponibles
    ),
    
    'Linear Regression': LinearRegression(
        fit_intercept=True,        # Estimer une ordonnée à l'origine
        normalize=False,           # Normaliser les données
        n_jobs=None                # Utilisation de tous les cœurs disponibles
    )
}

# Créer le scorer pour la MSE
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)

# Initialiser les variables pour stocker les résultats
best_model_name = None
best_composite_score = float('-inf')  # Le score composite à maximiser
best_model_results = {}

# Pondérations des différentes métriques (vous pouvez ajuster selon vos priorités)
weight_mse = 0.4  # Plus bas, mieux c'est
weight_rmse = 0.3  # Plus bas, mieux c'est
weight_r2 = 0.3    # Plus haut, mieux c'est

# Itérer sur les modèles
for model_name, model in models.items():
    # Validation croisée avec 10 splits
    cv_scores_mse = cross_val_score(model, X_encoded, y_encoded, cv=10, scoring=mse_scorer)
    # Calculer la moyenne de la MSE
    mean_mse = -cv_scores_mse.mean()
    std_mse = cv_scores_mse.std()
    # Calculer le RMSE
    rmse = sqrt(mean_mse)
    # Ajuster le modèle pour calculer le R² sur l'ensemble complet
    model.fit(X_encoded, y_encoded)
    y_pred = model.predict(X_encoded)
    r2 = r2_score(y_encoded, y_pred)
    # Calculer le score composite basé sur les pondérations
    composite_score = (-weight_mse * mean_mse) + (-weight_rmse * rmse) + (weight_r2 * r2)
    # Afficher les résultats pour chaque modèle
    print(f'{model_name}:')
    print(f'  Mean MSE from cross-validation: {mean_mse:.4f}')
    print(f'  Standard Deviation of MSE: {std_mse:.4f}')
    print(f'  RMSE: {rmse:.4f}')
    print(f'  R2 Score: {r2:.4f}')
    print(f'  Composite Score: {composite_score:.4f}\n')

    # Mettre à jour si ce modèle est le meilleur selon le score composite
    if composite_score > best_composite_score:
        best_composite_score = composite_score
        best_model_name = model_name
        best_model = model
        best_model_results = {
            'Mean MSE': mean_mse,
            'RMSE': rmse,
            'R2 Score': r2,
            'Composite Score': composite_score,
        }
# Afficher le meilleur modèle
print(f'Best Model: {best_model_name}')
print(f'Mean MSE: {best_model_results["Mean MSE"]:.4f}')
print(f'RMSE: {best_model_results["RMSE"]:.4f}')
print(f'R2 Score: {best_model_results["R2 Score"]:.4f}')
print(f'Composite Score: {best_model_results["Composite Score"]:.4f}')

Ridge:
  Mean MSE from cross-validation: 1.9313
  Standard Deviation of MSE: 1.0151
  RMSE: 1.3897
  R2 Score: 0.8252
  Composite Score: -0.9419

Lasso:
  Mean MSE from cross-validation: 2.0305
  Standard Deviation of MSE: 1.2419
  RMSE: 1.4249
  R2 Score: 0.8081
  Composite Score: -0.9972

XGBoost:
  Mean MSE from cross-validation: 2.1828
  Standard Deviation of MSE: 1.2528
  RMSE: 1.4774
  R2 Score: 1.0000
  Composite Score: -1.0164

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000324 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 127
[LightGBM] [Info] Number of data points in the train set: 359, number of used features: 13
[LightGBM] [Info] Start training from score 11.891365
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Light